# Validator: Supervisor Refactor & Tool Registry

This notebook validates the cognitive complexity reduction and the correct wiring of the `supervisor_node` following the Hexagonal Architecture refactor.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path to ensure imports work
project_root = str(Path(os.getcwd()).parent)
if project_root not in sys.path:
    sys.path.append(project_root)

# Reload modules to pick up changes
%load_ext autoreload
%autoreload 2

## 1. Validate Registry Bootstrapping
We check if `config.py` correctly factories the registry.

In [ ]:
from neuro_agent.domain.config import bootstrap_tool_registry

registry = bootstrap_tool_registry()
print(f"Tools loaded: {registry.list_tools()}")

# Check schema of a tool
web_search_tools = registry.get_bedrock_config()
import json
print(json.dumps(web_search_tools[0], indent=2))

## 2. Validate Supervisor Node Logic
We will run the `supervisor_node` with a mock state. 
**Note**: This will attempt to call AWS Bedrock. If you don't have credentials or the model `us.amazon.nova-pro-v1:0` enabled, it will error, but that *confirms* the code reached the integration point.

In [ ]:
from neuro_agent.apps.supervisor.nodes import supervisor_node
from langchain_core.messages import HumanMessage

# Mock State
state = {
    "messages": [HumanMessage(content="Find me information about Quantum Computing.")],
    "user_id": "validation_user_001",
    "profile": {"pref": "concise"},
    "todos": []
}

# Config injecting the registry
config = {
    "configurable": {
        "tool_registry": registry
    }
}

try:
    # Invocation
    result = supervisor_node(state, config)
    print("\n--- Result ---")
    print(result)
except Exception as e:
    # Anticipated errors: Credentials, Region, Model Access
    print(f"\nIntegration reached AWS Client but failed (Expected if no creds): {e}")